In [1]:
using Random
using JSON
include("readclassjson.jl")

data = readclassjson("feat_valid.json")

# Question 3
# Read in data from JSON file into U and v.
U = data["U"]
v = data["v"]
raw_data_embed = [U v]

# Split data into training and test sets
indices = shuffle(1:300)
train = raw_data_embed[indices[1:240] , 1:4]
test = raw_data_embed[indices[241:end], 1:4]


60×4 Array{Float64,2}:
  0.866546  -1.28893     0.2079       0.673225 
 -1.80303   -0.98377    -1.17554     31.8833   
  0.313382   0.384075    0.639474     0.387165 
 -0.645597   1.25317     1.06623     -3.83126  
  0.363839   1.4047      0.27181     15.2844   
  2.12999   -0.268714    0.56861     15.6798   
  0.378599  -0.577509    0.0175294    0.191602 
 -1.24361   -0.88967    -1.18121     15.7772   
  0.75606   -0.601307   -0.61301     -1.74961  
  0.542924   0.673145   -1.14139      0.890509 
  1.4405    -0.239126    1.34126     -1.79595  
 -2.63399   -0.367349   -0.213456    44.8091   
 -1.28258   -0.580084   -0.739417    14.5982   
  ⋮                                            
 -1.00886    0.16946     0.72822      0.889756 
  0.998677   0.208334    0.974627     2.54017  
 -0.629805   0.197449   -1.69487    -13.4237   
  1.73517   -1.77497     0.117025    -0.0572531
  0.644293   1.0301     -0.543327    12.5482   
  0.550852  -0.307905   -0.0677981    0.286118 
  0.111043   0.95

In [2]:
# Implement softest nearest neighbor
function softknn(X, Y, x, rho)
    n = size(X)[1]
    exp_weights = [exp(-sum((X[i,:] .- x).^2)/rho^2) for i=1:n]
    w = exp_weights / sum(exp_weights) #find weights
    y_hat = sum(w .* Y, dims=1) #weighted combination of Y
    return y_hat
end

softknn (generic function with 1 method)

In [3]:
using LinearAlgebra
# Test softest nearest neighbor
# Question 3a
tr_e1 = []
tr_e2 = []
tr_e3 = []
tr_e4 = []
# Calculate training errors
for j in 1:240
    tr_sk1 = softknn(train[:, 1:3], train[:, 4], train[j, 1:3], .25) 
    tr_sk2 = softknn(train[:, 1:3], train[:, 4], train[j, 1:3], .5) 
    tr_sk3 = softknn(train[:, 1:3], train[:, 4], train[j, 1:3], 1)
    tr_sk4 = softknn(train[:, 1:3], train[:, 4], train[j, 1:3], 2)
    push!(tr_e1, (tr_sk1[1] - train[j, 4]))
    push!(tr_e2, (tr_sk2[1] - train[j, 4]))
    push!(tr_e3, (tr_sk3[1] - train[j, 4]))
    push!(tr_e4, (tr_sk4[1] - train[j, 4]))
end

tr_rms1 = norm(tr_e1)/ sqrt(length(tr_e1))
tr_rms2 = norm(tr_e2)/ sqrt(length(tr_e2))
tr_rms3 = norm(tr_e3)/ sqrt(length(tr_e3))
tr_rms4 = norm(tr_e4)/ sqrt(length(tr_e4))



te_e1 = []
te_e2 = []
te_e3 = []
te_e4 = []
# Calculate test errors
for i in 1:60
    te_sk1 = softknn(train[:, 1:3], train[:, 4], test[i, 1:3], .25) 
    te_sk2 = softknn(train[:, 1:3], train[:, 4], test[i, 1:3], .5) 
    te_sk3 = softknn(train[:, 1:3], train[:, 4], test[i, 1:3], 1)
    te_sk4 = softknn(train[:, 1:3], train[:, 4], test[i, 1:3], 2)
    push!(te_e1, (te_sk1[1] - test[i, 4]))
    push!(te_e2, (te_sk2[1] - test[i, 4]))
    push!(te_e3, (te_sk3[1] - test[i, 4]))
    push!(te_e4, (te_sk4[1] - test[i, 4]))
end

te_rms1 = norm(te_e1)/ sqrt(length(te_e1))
te_rms2 = norm(te_e2)/ sqrt(length(te_e2))
te_rms3 = norm(te_e3)/ sqrt(length(te_e3))
te_rms4 = norm(te_e4)/ sqrt(length(te_e4))
# Report values for train rms errors for rho = [.25, .5, 1, 2]
# Report values for test rms errors for rho = [.25, .5, 1, 2]
@show tr_rms1
@show te_rms1
@show " "

@show tr_rms2
@show te_rms2
@show " "

@show tr_rms3
@show te_rms3
@show " "

@show tr_rms4
@show te_rms4


tr_rms1 = 0.47976217690814615
te_rms1 = 6.596192968565304
" " = " "
tr_rms2 = 1.9512720822475083
te_rms2 = 5.015807488109162
" " = " "
tr_rms3 = 7.011468665901686
te_rms3 = 8.352773079367262
" " = " "
tr_rms4 = 13.399421774836787
te_rms4 = 12.898930707836534


12.898930707836534

In [4]:
# Question 3b
# Test soft knn using new embedding function 
phi(U) = [U U[:,1].*U[:,2] U[:,2].*U[:,3] U[:,1].*U[:,3] U.^2]
new_embed = [phi(U) v]
new_train = new_embed[indices[1:240] , 1:10]
new_test = new_embed[indices[241:end], 1:10]
new_tr_e1 = []
new_tr_e2 = []
new_tr_e3 = []
new_tr_e4 = []
# Calculate new training errors
for k in 1:240
    new_tr_sk1 = softknn(new_train[:, 1:9], new_train[:, 10], new_train[k, 1:9], .25) 
    new_tr_sk2 = softknn(new_train[:, 1:9], new_train[:, 10], new_train[k, 1:9], .5) 
    new_tr_sk3 = softknn(new_train[:, 1:9], new_train[:, 10], new_train[k, 1:9], 1)
    new_tr_sk4 = softknn(new_train[:, 1:9], new_train[:, 10], new_train[k, 1:9], 2)
    push!(new_tr_e1, (new_tr_sk1[1] - new_train[k, 10]))
    push!(new_tr_e2, (new_tr_sk2[1] - new_train[k, 10]))
    push!(new_tr_e3, (new_tr_sk3[1] - new_train[k, 10]))
    push!(new_tr_e4, (new_tr_sk4[1] - new_train[k, 10]))
end

new_tr_rms1 = norm(new_tr_e1)/ sqrt(length(new_tr_e1))
new_tr_rms2 = norm(new_tr_e2)/ sqrt(length(new_tr_e2))
new_tr_rms3 = norm(new_tr_e3)/ sqrt(length(new_tr_e3))
new_tr_rms4 = norm(new_tr_e4)/ sqrt(length(new_tr_e4))



new_te_e1 = []
new_te_e2 = []
new_te_e3 = []
new_te_e4 = []
# Calculate test errors
for l in 1:60
    new_te_sk1 = softknn(new_train[:, 1:9], new_train[:, 10], new_test[l, 1:9], .25) 
    new_te_sk2 = softknn(new_train[:, 1:9], new_train[:, 10], new_test[l, 1:9], .5) 
    new_te_sk3 = softknn(new_train[:, 1:9], new_train[:, 10], new_test[l, 1:9], 1)
    new_te_sk4 = softknn(new_train[:, 1:9], new_train[:, 10], new_test[l, 1:9], 2)
    push!(new_te_e1, (new_te_sk1[1] - new_test[l, 10]))
    push!(new_te_e2, (new_te_sk2[1] - new_test[l, 10]))
    push!(new_te_e3, (new_te_sk3[1] - new_test[l, 10]))
    push!(new_te_e4, (new_te_sk4[1] - new_test[l, 10]))
end

new_te_rms1 = norm(new_te_e1)/ sqrt(length(new_te_e1))
new_te_rms2 = norm(new_te_e2)/ sqrt(length(new_te_e2))
new_te_rms3 = norm(new_te_e3)/ sqrt(length(new_te_e3))
new_te_rms4 = norm(new_te_e4)/ sqrt(length(new_te_e4))
# Report values for train rms errors for rho = [.25, .5, 1, 2]
# Report values for test rms errors for rho = [.25, .5, 1, 2]
@show new_tr_rms1
@show new_te_rms1
@show " "

@show new_tr_rms2
@show new_te_rms2
@show " "

@show new_tr_rms3
@show new_te_rms3
@show " "

@show new_tr_rms4
@show new_te_rms4

new_tr_rms1 = 0.13077815282588331
new_te_rms1 = 6.656834670475699
" " = " "
new_tr_rms2 = 0.4634914522532791
new_te_rms2 = 6.367705188953499
" " = " "
new_tr_rms3 = 1.9348265034806758
new_te_rms3 = 5.942271715281465
" " = " "
new_tr_rms4 = 5.477932600280858
new_te_rms4 = 8.19429761391233


8.19429761391233

In [ ]:
# Question 3c
# From the following tests, we see that the best performing 
# predictor is the k nearest neighbor algorithm with k = 2,
# which gives a test error of 5.015807488109162